## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-01-05-48-17 +0000,nypost,Florida A&M announcer under fire for remark ab...,https://nypost.com/2025/10/01/us-news/florida-...
1,2025-10-01-05-23-54 +0000,nypost,Missouri son accused to bludgeoning mother to ...,https://nypost.com/2025/10/01/us-news/missouri...
2,2025-10-01-05-19-00 +0000,wsj,Asia Manufacturing PMIs Show Diverging Impact ...,https://www.wsj.com/economy/asia-manufacturing...
3,2025-10-01-05-17-58 +0000,bbc,Students say 'unlimited' energy costing £2k more,https://www.bbc.com/news/articles/c75qkz9wr67o...
4,2025-10-01-05-14-09 +0000,nyt,U.S. Eases Visa Restrictions on South Korean W...,https://www.nytimes.com/2025/10/01/world/asia/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2373/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
187,trump,56
86,shutdown,29
85,government,26
7,new,18
94,deal,18


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
76,2025-10-01-00-39-00 +0000,wsj,The government was on course to shut down earl...,https://www.wsj.com/politics/policy/government...,151
337,2025-09-30-06-50-20 +0000,nypost,Trump mocks Chuck Schumer and Hakeem Jeffries ...,https://nypost.com/2025/09/30/us-news/trump-mo...,133
157,2025-09-30-20-20-33 +0000,nypost,Trump says Harvard will run new trade schools ...,https://nypost.com/2025/09/30/us-news/trump-an...,125
155,2025-09-30-20-27-00 +0000,wsj,President Trump hailed the use of military for...,https://www.wsj.com/politics/national-security...,124
175,2025-09-30-19-20-21 +0000,nypost,Trump threatens ‘irreversible’ actions if Dems...,https://nypost.com/2025/09/30/us-news/trump-th...,124


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
76,151,2025-10-01-00-39-00 +0000,wsj,The government was on course to shut down earl...,https://www.wsj.com/politics/policy/government...
13,89,2025-10-01-04-36-00 +0000,wsj,The White House and lawmakers failed to reach ...,https://www.wsj.com/politics/policy/government...
155,56,2025-09-30-20-27-00 +0000,wsj,President Trump hailed the use of military for...,https://www.wsj.com/politics/national-security...
275,55,2025-09-30-13-35-39 +0000,nyt,"With New U.S. Proposal to End Gaza War, a Rare...",https://www.nytimes.com/2025/09/30/world/middl...
82,40,2025-10-01-00-24-27 +0000,nypost,Michigan church attack victims include husband...,https://nypost.com/us-news/victims-of-michigan...
231,39,2025-09-30-15-50-00 +0000,bbc,Trump peace plan 'ignores interests of Palesti...,https://www.bbc.com/news/articles/cx2j97jldkmo...
217,34,2025-09-30-16-21-56 +0000,bbc,South Africa's top diplomat in France found de...,https://www.bbc.com/news/articles/ce3y505yq4ko...
157,33,2025-09-30-20-20-33 +0000,nypost,Trump says Harvard will run new trade schools ...,https://nypost.com/2025/09/30/us-news/trump-an...
311,31,2025-09-30-10-00-00 +0000,latimes,"22,500 homes lost. Over five years later, only...",https://www.latimes.com/homeless-housing/story...
124,31,2025-09-30-22-04-22 +0000,nypost,Ford CEO warns there’s not enough blue-collar ...,https://nypost.com/2025/09/30/business/ford-ce...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
